# BC 3
## Single tiny network; 
## Input = sub-regions 

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas as pd
import os
import random as rn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
SEED = 321
SCALE = 32
rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
NUM_TN = 5
SUB_REGION_SCALE = 14

In [4]:
data = "../input/flowers-recognition/flowers/"
folders = os.listdir(data)

In [5]:
image_names = []
train_labels = []
train_images = []

size = 32,32

for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            image_names.append(os.path.join(data,folder,file))
            train_labels.append(folder)
            img = cv2.imread(os.path.join(data,folder,file))
            im = cv2.resize(img,size)
            train_images.append(im)
        else:
            continue

In [6]:
train = np.array(train_images)
train = train.astype('float32')
train.shape

(4323, 32, 32, 3)

In [7]:
# Train, Val, Test split = 0.8, 0.1, 0.1 of the dataset
X_train,X_val,y_train,y_val = train_test_split(train,train_labels, test_size = 0.2)
X_val,X_test,y_val,y_test = train_test_split(X_val,y_val, test_size = 0.5)

In [8]:
#Generate cropped train image
X_train_cropped = []
y_train_cropped = []
for i in range (X_train.shape[0]):
    image_cropped = tf.image.random_crop(X_train[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE, 3])
    image_cropped = np.array(image_cropped)
    X_train_cropped.append(image_cropped)
    y_train_cropped.append(y_train[i])
    
X_train_cropped = np.array(X_train_cropped)
y_train_cropped = np.array(y_train_cropped)

In [9]:
#Generate cropped test image
X_test_cropped_list = []
y_test_cropped_list = []

for i in range(NUM_TN):
    X_test_cropped = []
    y_test_cropped = []
    for i in range (X_test.shape[0]):
        image_cropped = tf.image.random_crop(X_test[i], size=[SUB_REGION_SCALE, SUB_REGION_SCALE, 3])
        image_cropped = np.array(image_cropped)
        X_test_cropped.append(image_cropped)
        y_test_cropped.append(y_test[i])
    X_test_cropped = np.array(X_test_cropped)
    y_test_cropped = np.array(y_test_cropped)

    X_test_cropped_list.append(X_test_cropped)
    y_test_cropped_list.append(y_test_cropped)

In [10]:
#normalizing dataset
X_train_cropped = X_train_cropped/255

for i in range(NUM_TN):
  X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [11]:
#transform to vector
X_train_cropped = X_train_cropped.reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))

for i in range(NUM_TN):
  X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE*3))

In [12]:
#one-hot encoding
y_train_cropped = pd.get_dummies(y_train_cropped)
y_train_cropped = y_train_cropped.values.argmax(1)
y_train_cropped = to_categorical(y_train_cropped)

for i in range(NUM_TN):
    y_test_cropped_list[i] = pd.get_dummies(y_test_cropped_list[i])
    y_test_cropped_list[i] = y_test_cropped_list[i].values.argmax(1)
    y_test_cropped_list[i] = to_categorical(y_test_cropped_list[i])
    

In [13]:
#build ANN model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE*3))
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation = 'softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
#train model
hist = model.fit(X_train_cropped, y_train_cropped, validation_data=(X_test_cropped_list[0], y_test_cropped_list[0]), epochs=5, batch_size=128)

Epoch 1/5
28/28 [==============================] - 2s 22ms/step - loss: 1.5684 - accuracy: 0.2759 - val_loss: 1.5131 - val_accuracy: 0.3372
Epoch 2/5
28/28 [==============================] - 0s 4ms/step - loss: 1.4266 - accuracy: 0.3519 - val_loss: 1.4239 - val_accuracy: 0.3141
Epoch 3/5
28/28 [==============================] - 0s 4ms/step - loss: 1.3962 - accuracy: 0.3742 - val_loss: 1.4169 - val_accuracy: 0.3418
Epoch 4/5
28/28 [==============================] - 0s 4ms/step - loss: 1.3990 - accuracy: 0.3803 - val_loss: 1.4079 - val_accuracy: 0.3857
Epoch 5/5
28/28 [==============================] - 0s 4ms/step - loss: 1.3697 - accuracy: 0.3870 - val_loss: 1.4460 - val_accuracy: 0.3233


## Model Evaluation

In [15]:
# Predict on test image
y_pred_list = []
for i in range(NUM_TN):
  y_pred = model.predict(X_test_cropped_list[i])
  y_pred_list.append(y_pred)
    
# Ensemble voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.argmax(np.sum(y_pred_list, axis=0), axis=1)

In [16]:
y_test_label_encoded_list =[]

In [17]:
for k in y_test_cropped_list:
    y_test_label_encoded_list.append(k)
y_test_label_encoded_list  = np.array(y_test_label_encoded_list)

In [18]:
y_test_label_encoded_list = np.argmax(np.sum(y_test_label_encoded_list, axis=0), axis=1)

In [19]:
accuracy = accuracy_score(y_test_label_encoded_list, y_pred_list)
accuracy

0.3833718244803695